In [1]:
import pickle
with open('ROMS_26yr_daydic.pkl', 'rb') as file:
    daydic = pickle.load(file)

with open('ROMS_26yr_eddies.pkl', 'rb') as file:
    eddies = pickle.load(file)

In [2]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

fname = '/srv/scratch/z3533156/26year_BRAN2020/outer_avg_01461.nc'
dataset = nc.Dataset(fname)
lon_rho  = dataset.variables['lon_rho']
lon_rho = np.transpose(lon_rho, axes=(1, 0))
lat_rho  = dataset.variables['lat_rho']
lat_rho = np.transpose(lat_rho, axes=(1, 0))
z_r = np.load('/home/z5297792/MRes/z_r.npy')
z_r = np.transpose(z_r, (1, 2, 0))
h = dataset.variables['h']
h = np.transpose(h, (1, 0))


TOTAL_NUM_DAYS = len(daydic)

## Tilt Properties - Ensure they're using the same depth range

In [3]:
def pav(xx, yy, zz):

    d_range = (-1000 <= zz) & (zz <= -200)

    xx = xx[d_range]
    yy = yy[d_range]
    zz = zz[d_range]

    centroid = np.mean(np.column_stack((xx, yy, zz)), axis=0)
    mean_centered_data = np.column_stack((xx - centroid[0], yy - centroid[1], zz - centroid[2]))
    cov_matrix = np.cov(mean_centered_data, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    principal_axis = sorted_eigenvectors[:, 0]
    
    if principal_axis[2] < 0:
        principal_axis = -principal_axis

    df = pd.DataFrame({
        'x': [centroid[0], principal_axis[0]],
        'y': [centroid[1], principal_axis[1]],
        'z': [centroid[2], principal_axis[2]]
    }, index=['Centroid', 'Principal Axis'])
    
    return df

def true_bearing(x, y):
    angle_rad = math.atan2(y, x)
    angle_deg = math.degrees(angle_rad)
    true_bearing = (90 - angle_deg) % 360
    return true_bearing

def tilt_distance(d_vec, z_btm):

    if z_btm <= -1000:
        z_btm = -1000
    z_top = -200

    TD = (z_top-z_btm)/d_vec[2]*np.sqrt(d_vec[0]**2+d_vec[1]**2)

    return TD

def calculate_r_squared(x, y, z, point_on_line, direction_vector):
    
    d_range = (-1000 <= z) & (z <= -200)

    x = x[d_range]
    y = y[d_range]
    z = z[d_range]

    def perpendicular_distances(x_coords, y_coords, z_coords, point_on_line, direction_vector):
        Q = np.array(point_on_line)
        d = np.array(direction_vector)
        direction_vector_magnitude = np.linalg.norm(d)
        distances = []
        for x, y, z in zip(x_coords, y_coords, z_coords):
            P = np.array([x, y, z])
            PQ = P - Q
            cross_product = np.cross(PQ, d)
            cross_product_magnitude = np.linalg.norm(cross_product)
            distance = cross_product_magnitude / direction_vector_magnitude
            distances.append(distance)
        return distances

    TSS = np.sum((np.array(x)-np.mean(x))**2 + (np.array(y)-np.mean(y))**2 +(np.array(z)-np.mean(z))**2)
    SSD = np.sum(np.array(perpendicular_distances(x, y, z, point_on_line, direction_vector))**2)
    r_squared = 1 - SSD/TSS

    return r_squared

def distance(lat1, lon1, lat2, lon2):
    EARTH_RADIUS = 6357000  # in meters
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2.0)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return EARTH_RADIUS * c

In [4]:
# def cow(eddy):
    
#     SEARCH_RADIUS_MAX = 40000
#     WIDTH, LENGTH = lon_rho.shape

#     RATIO = 2/3
#     calc_depths = z_r[150, 150, :]

#     brngs = []
#     TDs = []
#     days_tilt_measured = []

#     previous_file_idx = -1    
#     for t in range(TOTAL_NUM_DAYS):

#         if 'Day'+str(t+1) in eddies[eddy].index:

#             eddy_at_surface = [eddies[eddy].loc['Day' + str(t + 1)].Lon, eddies[eddy].loc['Day' + str(t + 1)].Lat]
#             PREVIOUS_CENTER = eddy_at_surface

#             R = np.sqrt((lon_rho - eddy_at_surface[0])**2 + (lat_rho - eddy_at_surface[1])**2)
#             i_search, j_search = np.argwhere(R == np.min(R))[0]
        
#             theta0 = np.deg2rad(lon_rho[i_search, j_search])
#             phi0 = np.deg2rad(lat_rho[i_search, j_search])

#             vert_centers = []
#             brng = []
#             TD = []

#             t_mod = t % 30
#             file_idx = np.floor(t/30)
#             if file_idx != previous_file_idx:
#                 fnumber = str(round(1461 + file_idx * 30)).zfill(5)
#                 ow = np.load('/srv/scratch/z5297792/Climatology/ow_and_cur_speeds/ow_'+fnumber+'.npy')
#                 cur_speed = np.load('/srv/scratch/z5297792/Climatology/ow_and_cur_speeds/cur_speed_'+fnumber+'.npy')
#                 previous_file_idx = file_idx

#             for i, depth in enumerate(calc_depths):

#                 SEARCH_RADIUS = 5000
#                 potential = {}
#                 flag = 1
#                 while flag:
        
#                     ratio = 0
#                     R = distance(PREVIOUS_CENTER[1], PREVIOUS_CENTER[0], lat_rho, lon_rho)
#                     ic, jc = np.where(R < SEARCH_RADIUS)
                    
#                     ow_lvl = np.full((WIDTH, LENGTH), np.nan)
#                     cur_speed_lvl = np.full((WIDTH, LENGTH), np.nan)
                    
#                     ow_lvl[ic, jc] = ow[ic, jc, i, t_mod]
#                     cur_speed_lvl[ic, jc] = cur_speed[ic, jc, i, t_mod]
                    
#                     neg_ow_indices = set(zip(*np.where(ow_lvl < -2E-11))) # -2E-11
#                     crit_cur_speed_indices = set(zip(*np.where(cur_speed_lvl < 0.05)))
                    
#                     potential = neg_ow_indices & crit_cur_speed_indices

#                     if len(ic) != 0:
#                         ratio = len(potential)/len(ic)
#                     else:
#                         ratio = 0
                    
#                     if potential or depth >= -6:
#                         if ratio <= RATIO and ratio != 0:

#                             X = [(lon_rho[p[0], p[1]], lat_rho[p[0], p[1]]) for p in potential]
#                             C = np.mean(X, axis=0)
#                             PREVIOUS_CENTER = C
#                             vert_centers.append((C[0], C[1]))
#                             flag = 0 
#                         elif SEARCH_RADIUS <= SEARCH_RADIUS_MAX:
#                             SEARCH_RADIUS += 5000
#                         else:
#                             vert_centers.append((np.nan, np.nan))
#                             flag = 0
#                     else:
#                         flag = 0

#             lon = [tup[0] for tup in vert_centers]
#             lat = [tup[1] for tup in vert_centers]
            
#             lon_series = pd.Series(lon)
#             lat_series = pd.Series(lat)
            
#             first_non_nan = lon_series.first_valid_index()
#             last_non_nan = lon_series.last_valid_index()
            
#             if first_non_nan is not None and last_non_nan is not None:
#                 lon_series = lon_series[first_non_nan:last_non_nan].interpolate()
#                 lat_series = lat_series[first_non_nan:last_non_nan].interpolate()

#                 calculated_depths = calc_depths[first_non_nan:last_non_nan]

#                 if any(calculated_depths<=-400): 

#                     d_range = (-1000 <= calculated_depths) & (calculated_depths <= -200)
#                     pav_depths = calculated_depths[d_range]

#                     lon = lon_series.tolist()
#                     lat = lat_series.tolist()
                    
#                     if len(pav_depths) > 1:
#                         lonr = np.deg2rad(lon)
#                         latr = np.deg2rad(lat)
#                         theta0 = np.nanmean(lonr)
#                         phi0 = np.nanmean(latr)
#                         r_earth = 6371000
                        
#                         x = r_earth * np.cos(phi0) * (lonr - theta0)
#                         y = r_earth * (latr - phi0)
                        
#                         if any(calculated_depths <= -400):
                            
#                             PAV = pav(x, y, calculated_depths)
#                             brng = true_bearing(PAV.loc['Principal Axis'].x, PAV.loc['Principal Axis'].y)
#                             d_vec = (PAV.loc['Principal Axis'].x, PAV.loc['Principal Axis'].y, PAV.loc['Principal Axis'].z)
#                             z_btm = calculated_depths[-1]
#                             TD = tilt_distance(d_vec, z_btm)
#                             R2 = calculate_r_squared(x, y, calculated_depths, PAV.loc['Centroid'], PAV.loc['Principal Axis'])

#                             if TD < 500000 and R2 >= .9:

#                                 brngs.append(brng)
#                                 TDs.append(TD)
#                                 days_tilt_measured.append(t+1)


#     days_to_keep = ['Day{}'.format(i) for i in days_tilt_measured]

#     eddies[eddy] = eddies[eddy].loc[days_to_keep]

#     eddies[eddy]['Tilt Bearing'] = brngs
#     eddies[eddy]['Tilt Distance'] = TDs





In [5]:
def cow(eddy):
    
    SEARCH_RADIUS_MAX = 50000
    WIDTH, LENGTH = lon_rho.shape
    CURSPEED_THRESH = 0.05
    OW_THRESH = -2e-11
    RATIO_THRESH = 2/3
    
    calc_depths = z_r[150, 150, :]

    brngs = []
    TDs = []
    days_tilt_measured = []

    previous_file_idx = -1    
    for t in range(TOTAL_NUM_DAYS):

        if 'Day'+str(t+1) in eddies[eddy].index:

            eddy_at_surface = [eddies[eddy].loc['Day' + str(t + 1)].Lon, eddies[eddy].loc['Day' + str(t + 1)].Lat]
            PREVIOUS_CENTER = eddy_at_surface

            R = np.sqrt((lon_rho - eddy_at_surface[0])**2 + (lat_rho - eddy_at_surface[1])**2)
            i_search, j_search = np.argwhere(R == np.min(R))[0]
        
            theta0 = np.deg2rad(lon_rho[i_search, j_search])
            phi0 = np.deg2rad(lat_rho[i_search, j_search])

            vert_centers = []
            brng = []
            TD = []

            t_mod = t % 30
            file_idx = np.floor(t/30)
            if file_idx != previous_file_idx:
                fnumber = str(round(1461 + file_idx * 30)).zfill(5)
                ow = np.load('/srv/scratch/z5297792/Climatology/ow_and_cur_speeds/ow_'+fnumber+'.npy')
                cur_speed = np.load('/srv/scratch/z5297792/Climatology/ow_and_cur_speeds/cur_speed_'+fnumber+'.npy')
                previous_file_idx = file_idx

            for i, depth in enumerate(calc_depths):

                SEARCH_RADIUS = 5000
                potential = {}
                flag = 1
                while flag:
        
                    R = distance(PREVIOUS_CENTER[1], PREVIOUS_CENTER[0], lat_rho, lon_rho)
                    mask = R < SEARCH_RADIUS
                
                    cur_speed_masked = np.where(mask, cur_speed[:,:,i, t_mod], np.nan) 
                    ow_masked = np.where(mask, ow[:,:,i, t_mod], np.nan) 
                
                    pot_i, pot_j = np.where((cur_speed_masked < CURSPEED_THRESH) & (ow_masked < OW_THRESH))

                    if len(pot_i) != 0:
                        ratio = len(pot_i)/sum(sum(mask))
                    else:
                        ratio = 0
                    
                    if len(pot_i) > 0 or depth >= -6:
                        if ratio <= RATIO_THRESH and ratio != 0:
                            X = [lon_rho[pot_i, pot_j], lat_rho[pot_i, pot_j]]
                            C = np.mean(X, axis=1)
                            PREVIOUS_CENTER = C
                            vert_centers.append((C[0], C[1]))
                            flag = 0 
                        elif SEARCH_RADIUS <= SEARCH_RADIUS_MAX:
                            SEARCH_RADIUS += 5000
                        else:
                            vert_centers.append((np.nan, np.nan))
                            flag = 0
                    else:
                        flag = 0

            lon = [tup[0] for tup in vert_centers]
            lat = [tup[1] for tup in vert_centers]
            
            lon_series = pd.Series(lon)
            lat_series = pd.Series(lat)
            
            first_non_nan = lon_series.first_valid_index()
            last_non_nan = lon_series.last_valid_index()
            
            if first_non_nan is not None and last_non_nan is not None:
                lon_series = lon_series[first_non_nan:last_non_nan].interpolate()
                lat_series = lat_series[first_non_nan:last_non_nan].interpolate()

                calculated_depths = calc_depths[first_non_nan:last_non_nan]

                if any(calculated_depths<=-400): 

                    d_range = (-1000 <= calculated_depths) & (calculated_depths <= -200)
                    pav_depths = calculated_depths[d_range]

                    lon = lon_series.tolist()
                    lat = lat_series.tolist()
                    
                    if len(pav_depths) > 1:
                        lonr = np.deg2rad(lon)
                        latr = np.deg2rad(lat)
                        r_earth = 6371000
                        
                        x = r_earth * np.cos(phi0) * (lonr - theta0)
                        y = r_earth * (latr - phi0)
                        
                        if any(calculated_depths <= -400):
                            
                            PAV = pav(x, y, calculated_depths)
                            brng = true_bearing(PAV.loc['Principal Axis'].x, PAV.loc['Principal Axis'].y)
                            d_vec = (PAV.loc['Principal Axis'].x, PAV.loc['Principal Axis'].y, PAV.loc['Principal Axis'].z)
                            z_btm = calculated_depths[-1]
                            TD = tilt_distance(d_vec, z_btm)
                            R2 = calculate_r_squared(x, y, calculated_depths, PAV.loc['Centroid'], PAV.loc['Principal Axis'])

                            if TD < 500000 and R2 >= .9:

                                brngs.append(brng)
                                TDs.append(TD)
                                days_tilt_measured.append(t+1)


    days_to_keep = ['Day{}'.format(i) for i in days_tilt_measured]

    eddies[eddy] = eddies[eddy].loc[days_to_keep]

    eddies[eddy]['Tilt Bearing'] = brngs
    eddies[eddy]['Tilt Distance'] = TDs





In [6]:
eddy_sample = [f"Eddy{i}" for i in range(14745,14745+1)] #for 5yr ~2800 eddies #for 26yr 14745 eddies

In [7]:
# EDDY = 'Eddy123'
time_check = 1
for EDDY in eddy_sample:
    cow(EDDY)
    if time_check % 100 == 0:
        print(EDDY)
    time_check += 1

In [8]:
with open('/home/z5297792/MRes/Climatology/ROMS_26yr_COWed_eddies_FIXED_E14745.pkl', 'wb') as pickle_file:
    pickle.dump(eddies, pickle_file)

In [ ]:
print('Complete!')